In [1]:
from scipy import stats
import random
import time
import matplotlib.pyplot as plt
import pylob as lob
from IPython.display import clear_output

In [2]:
def generate_orders(n: int):
    result = list()
    
    for _ in range(n):
    
        midprice = 110_000
    
        n_ask_limits = stats.poisson.rvs(1000)
        n_bid_limits = stats.poisson.rvs(1000)
    
        ask_limits_price = stats.expon.rvs(loc=midprice, scale=1, size=n_ask_limits)
        bid_limits_price = -stats.expon.rvs(loc=midprice, scale=1, size=n_bid_limits) + 2*midprice
    
        ask_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_ask_limits)
        bid_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_bid_limits)
    
        ask_limits_params = [lob.OrderParams(lob.OrderSide.ASK, p, q) for (p, q) in zip(ask_limits_price, ask_limits_quantities)]
        bid_limits_params = [lob.OrderParams(lob.OrderSide.BID, p, q) for (p, q) in zip(bid_limits_price, bid_limits_quantities)]
    
        n_markets = stats.poisson.rvs(100)
    
        markets_price = stats.norm.rvs(loc=midprice, scale=2, size=n_markets)
        markets_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_markets)
        markets_bid_or_ask = [random.choice((lob.OrderSide.BID, lob.OrderSide.ASK)) for _ in range(n_markets)]
    
        markets_params = [lob.OrderParams(s, p, q) for (s, p, q) in zip(markets_bid_or_ask, markets_price, markets_quantities)]
    
        orders = ask_limits_params + bid_limits_params + markets_params
        random.shuffle(orders)
        
        result.append(orders)
        
    return result

In [3]:
def simulate(orders):
    ob = lob.OrderBook('Simulation')

    for o in orders:
        ob.process_many(o)
        print(ob.view())
        time.sleep(0.7)
        clear_output(wait=True)

In [4]:
orders = generate_orders(10)
simulate(orders)

   [ORDER-BOOK Simulation]

   ...(505 more asks)
 - 110000.10 | 074 | 03866.14 | 425275786.6140
 - 110000.09 | 080 | 04182.46 | 460070976.4214
 - 110000.08 | 081 | 04230.35 | 465338838.4280
 - 110000.07 | 098 | 04565.04 | 502154719.5528
 - 110000.06 | 089 | 04587.21 | 504593375.2326
 - 110000.05 | 107 | 05538.45 | 609229776.9225
 - 110000.04 | 088 | 04306.09 | 473670072.2436
 - 110000.03 | 064 | 03437.46 | 378120703.1238
 - 110000.02 | 010 | 00509.31 | 56024110.1862
 - 110000.00 | 001 | 00029.07 | 3197700.0000
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 - 109999.98 | 008 | 00441.98 | 48617791.1604
 - 109999.97 | 054 | 02658.46 | 292430520.2462
 - 109999.96 | 078 | 03368.25 | 370507365.2700
 - 109999.95 | 084 | 04612.49 | 507373669.3755
 - 109999.94 | 087 | 04262.61 | 468886844.2434
 - 109999.93 | 097 | 04866.81 | 535348759.3233
 - 109999.92 | 100 | 05300.21 | 583022675.9832
 - 109999.91 | 092 | 04448.10 | 489290599.6710
 - 109999.90 | 091 | 04639.59 | 510354436.0410
 - 109999.89 | 